# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

### Questão 1.1 

In [4]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'

In [5]:
# 2) Seu código aqui
r = requests.get(url)

In [6]:
r.status_code

200

In [7]:
data_json = r.json()

In [8]:
print(data_json)

{'items': [{'id_pleito': 10002, 'tipo_interessado': 'Estado', 'interessado': 'Rio de Janeiro', 'cod_ibge': 33, 'uf': 'RJ', 'num_pvl': None, 'status': 'Encaminhado à PGFN com manifestação técnica favorável', 'num_processo': '17944.001097/2009-67', 'data_protocolo': '2009-09-09T03:00:00Z', 'tipo_operacao': 'Operação contratual externa (com garantia da União)', 'finalidade': 'Profisco', 'tipo_credor': 'Instituição Financeira Internacional', 'credor': 'Banco Interamericano de Desenvolvimento', 'moeda': 'Dólar dos EUA', 'valor': 19759050, 'pvl_assoc_divida': 1, 'pvl_contradado_credor': 0, 'data_status': '28/09/2009'}, {'id_pleito': 8556, 'tipo_interessado': 'Estado', 'interessado': 'Rio de Janeiro', 'cod_ibge': 33, 'uf': 'RJ', 'num_pvl': None, 'status': 'Encaminhado à PGFN com manifestação técnica favorável', 'num_processo': '17944.000797/2010-78', 'data_protocolo': '2011-06-30T03:00:00Z', 'tipo_operacao': 'Operação contratual externa (com garantia da União)', 'finalidade': 'Multissetorial'

In [9]:
df = pd.DataFrame(data_json['items'])


In [ ]:
df.head(10)

### Questão 1.2

In [10]:
# Calcule a contagem de cada código de status
status_counts = df['status'].value_counts()

# Classifique as contagens em ordem decrescente e pegue os três primeiros
top_3_status= status_counts.head(3)
print(top_3_status)


status
Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Name: count, dtype: int64


### Questão 1.3

In [11]:
data_status = df['data_status']
data_status


0     28/09/2009
1     04/07/2011
2     13/07/2006
3     03/09/2012
4     04/07/2011
5     25/11/2009
6     04/07/2011
7     27/06/2002
8     04/10/2012
9     26/02/2016
10    28/08/2012
11    01/04/2013
12    05/02/2014
13    09/12/2009
14    14/05/2012
15    10/07/2012
16    28/03/2013
17    14/09/2012
18    06/03/2008
19    27/06/2002
20    18/06/2009
21    02/07/2012
22    20/04/2010
23    01/09/2014
24    01/09/2014
25    20/11/2012
26    21/11/2013
27    06/05/2013
28    29/07/2013
29    29/05/2012
30    06/05/2013
31    22/05/2009
32    27/12/2013
33    27/06/2002
34    27/12/2011
35    04/07/2014
36    19/08/2010
37    24/07/2009
38    07/06/2011
39    10/07/2012
40    06/07/2010
41    25/10/2016
42    04/11/2009
43    04/07/2011
44    11/03/2016
45    07/07/2009
46    18/04/2008
47    06/08/2009
48    04/10/2012
49    16/05/2016
50    27/06/2002
51    01/03/2006
Name: data_status, dtype: object

In [12]:
df['data_status'] = df['data_status'].astype(str)


# Dividir a coluna com base no padrão '/1' e pegar o terceiro elemento
ano = df['data_status'].str.split('/').str[2]
frequencia = ano.value_counts()
print(frequencia)

data_status
2012    11
2009     9
2013     7
2011     6
2002     4
2016     4
2014     4
2010     3
2006     2
2008     2
Name: count, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [13]:
#1) Seu código aqui
def consulta(uf: str, tipo: str):
    url= f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo}'
    r = requests.get(url)
    data_json = r.json()
    df = pd.DataFrame(data_json['items'])
    return df




In [14]:
# 2) Seu código aqui
solicitacoes_MG = consulta('MG', 'Estado')
solicitacoes_MG 

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,13587,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000031/2004-78,2004-11-27T02:00:00Z,Operação contratual externa (com garantia da U...,Desenvolvimento rural,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,3.500000e+07,1,0,09/12/2004
1,13672,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000048/2005-14,2005-12-28T02:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,1.700000e+08,1,0,19/04/2006
2,13695,Estado,Minas Gerais,31,MG,None,Arquivado,19405.000053/2003-57,2003-08-25T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,3.500000e+07,0,0,20/02/2004
3,20458,Estado,Minas Gerais,31,MG,00000.000000/2020-68,Arquivado a pedido,17944.001085/2015-81,2015-11-23T02:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,5.000000e+07,0,0,29/04/2016
4,9197,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000924/2009-03,2010-03-10T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,5.000000e+07,1,0,01/04/2010
5,22380,Estado,Minas Gerais,31,MG,00000.000000/2031-07,Arquivado pela STN,17944.000547/2016-23,2016-12-21T02:00:00Z,Operação contratual interna (com garantia da U...,Aditivo contratual,Instituição Financeira Nacional,Banco do Brasil S/A,Real,6.985602e+08,0,0,16/10/2017
6,5582,Estado,Minas Gerais,31,MG,None,Arquivado a pedido,17944.000148/2013-10,2013-04-02T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Corporação Andina de Fomento,Dólar dos EUA,3.000000e+08,0,0,10/04/2013
7,13447,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000002/2006-78,2009-02-02T02:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,1.000000e+07,1,0,12/11/2009
8,27063,Estado,Minas Gerais,31,MG,PVL02.002572/2017-60,Arquivado pela STN,17944.103474/2017-10,2020-06-10T13:12:13Z,Operação contratual interna,Renegociação de dívidas,União,União,Real,9.396517e+10,0,0,04/05/2021
9,5967,Estado,Minas Gerais,31,MG,None,Deferido,17944.000246/2011-95,2011-05-02T03:00:00Z,Concessão de garantia,Concessão de garantia a empresa estatal não de...,Instituição Financeira Internacional,Kreditanstalt für Wiederaufbau,Euro,1.000000e+08,0,0,19/05/2011


In [15]:
arquivado = df[ df['status']== 'Arquivado por decurso de prazo']
arquivado.shape[0]

1

In [16]:
# 3) Seu código aqui
solicitacoes_BA = consulta('BA', 'Município')

In [17]:
solicitacoes_BA 

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,6771,Município,Baianópolis,2902500,BA,None,Deferido,17944.000426/2008-71,2008-02-29T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,500000.00,0,0,12/03/2008
1,9402,Município,Nilo Peçanha,2922607,BA,None,Pendente de regularização,17944.000973/2012-33,2013-08-06T03:00:00Z,Operação contratual interna,Regularização de Dívida - Água e Esgoto,Empresa Estatal,Empresa Baiana de Águas e Saneamento S/A,Real,245894.72,0,0,09/06/2014
2,7687,Município,Sebastião Laranjeiras,2930006,BA,None,Arquivado a pedido,17944.000634/2012-57,2015-09-11T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,2000000.00,0,0,25/09/2015
3,7713,Município,Nordestina,2922656,BA,None,Deferido,17944.000639/2011-07,2011-07-25T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,400000.00,0,0,24/08/2011
4,7371,Município,Vitória da Conquista,2933307,BA,None,Arquivado por decurso de prazo,17944.000556/2011-18,2011-10-04T03:00:00Z,Operação contratual interna,Provias 2010,Instituição Financeira Nacional,Banco do Brasil S/A,Real,3000000.00,0,0,16/01/2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,11509,Município,Itagibá,2915205,BA,None,Deferido,17944.001464/2014-90,2014-09-23T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,3500000.00,0,0,06/10/2014
961,10436,Município,Canudos,2906824,BA,None,Arquivado por decurso de prazo,17944.001199/2014-40,2014-08-04T03:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,65000.00,0,0,18/02/2015
962,33916,Município,Amargosa,2901007,BA,PVL02.002925/2019-93,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,652635.69,1,1,22/11/2019
963,11521,Município,Conde,2908606,BA,None,Deferido,17944.001467/2007-02,2007-10-25T02:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,500000.00,0,0,12/11/2007


In [18]:
municipios_BA = solicitacoes_BA['interessado'].value_counts()

In [19]:
municipios_BA.head(1)

interessado
Camaçari    19
Name: count, dtype: int64

In [20]:
# 4) Seu código aqui
nome_arquivo = 'solicitacoes_bahia.csv'



In [21]:
solicitacoes_BA_estado = consulta('BA', 'Estado')

In [22]:
solicitacoes_BA_estado.to_csv(nome_arquivo, index = False)